## **Import Libraries**

In [2]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='pcM9bZAg6neNNB22w0lpBKZywHu92L8cT5YAn1vjXq6N',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-6vckwkjf6vbdwg'
object_key = 'Car damage.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


## **Unzipping Dataset**

In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)
    

In [5]:
import os
os.listdir('.')

['Car damage', '.virtual_documents']

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

## **MODEL FOR BODY TYPE DETECTION**

In [7]:
trainPath = '/home/wsuser/work/Car damage/body/training'
testPath = '/home/wsuser/work/Car damage/body/validation'

In [8]:

training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [9]:
training_set.class_indices

{'00-front': 0, '01-rear': 1, '02-side': 2}

## **Declaring Model Variable**

In [10]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                              


## **Compiling the Model**

In [11]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

## **Training model**

In [12]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/tmp/wsuser/ipykernel_164/289406290.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
97/97 [==============================] - 455s 5s/step - loss: 1.1211 - acc: 0.5593 - val_loss: 1.3210 - val_acc: 0.5706
Epoch 2/25
97/97 [==============================] - 424s 4s/step - loss: 0.6143 - acc: 0.7678 - val_loss: 0.8840 - val_acc: 0.6412
Epoch 3/25
97/97 [==============================] - 439s 5s/step - loss: 0.6095 - acc: 0.7781 - val_loss: 0.8494 - val_acc: 0.6412
Epoch 4/25
97/97 [==============================] - 440s 5s/step - loss: 0.3757 - acc: 0.8462 - val_loss: 1.0902 - val_acc: 0.6824
Epoch 5/25
97/97 [==============================] - 437s 5s/step - loss: 0.2461 - acc: 0.9154 - val_loss: 1.0222 - val_acc: 0.6471
Epoch 6/25
97/97 [==============================] - 435s 4s/step - loss: 0.1902 - acc: 0.9329 - val_loss: 1.0736 - val_acc: 0.6941
Epoch 7/25
97/97 [==============================] - 430s 4s/step - loss: 0.1538 - acc: 0.9505 - val_loss: 0.8297 - val_acc: 0.7059
Epoch 8/25
97/97 [==============================] - 449s 5s/step - loss: 0.1576 - a

In [13]:
model.save('body.h5')

In [14]:
!tar -zcvf body.tgz body.h5

body.h5


In [15]:
ls -1 

body.h5
body.tgz
'Car damage'/


In [16]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 17.3 MB/s eta 0:00:01


## **Connecting with IBM CLOUD**

In [20]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {"url":"https://us-south.ml.cloud.ibm.com", "apikey":"qntYN2kHZkW-YH94VfcclLtMCSKV2TFTO8Qmm3KR8JQK"}
client = APIClient(wml_credentials)

In [21]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [23]:
space_uid = guid_from_space_name(client, 'image classification')
#space_uid

In [24]:
client.set.default_space(space_uid)

'SUCCESS'

In [25]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
#software_spec_uid

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [27]:
model_details = client.repository.store_model(model = 'body.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "body", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [28]:
client.repository.download(model_id, 'body_cloud.tar.gz')

Successfully saved model content to file: 'body_cloud.tar.gz'


'/home/wsuser/work/body_cloud.tar.gz'

In [30]:
model_body = load_model('body.h5')

## **MODEL FOR LEVEL TYPE DETECTION**

In [33]:
trainPath = '/home/wsuser/work/Car damage/level/training'
testPath = '/home/wsuser/work/Car damage/level/validation'

In [34]:
training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [35]:
training_set.class_indices

{'01-minor': 0, '02-moderate': 1, '03-severe': 2}

## **Declaring Model Variable**

In [36]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model1=Model(inputs=vgg.input,outputs=prediction)
model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

In [37]:
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

## **Training model**

In [ ]:
y = model1.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/tmp/wsuser/ipykernel_164/2253460449.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  y = model1.fit_generator(


Epoch 1/25
97/97 [==============================] - 431s 4s/step - loss: 1.2081 - acc: 0.5604 - val_loss: 1.2990 - val_acc: 0.5882
Epoch 2/25
97/97 [==============================] - 429s 4s/step - loss: 0.7170 - acc: 0.7203 - val_loss: 0.8731 - val_acc: 0.6353
Epoch 3/25
97/97 [==============================] - 425s 4s/step - loss: 0.4918 - acc: 0.8091 - val_loss: 0.8918 - val_acc: 0.6647
Epoch 4/25
97/97 [==============================] - 426s 4s/step - loss: 0.4436 - acc: 0.8246 - val_loss: 1.9206 - val_acc: 0.5000
Epoch 5/25
97/97 [==============================] - 429s 4s/step - loss: 0.3659 - acc: 0.8545 - val_loss: 1.1350 - val_acc: 0.6294
Epoch 6/25
97/97 [==============================] - 428s 4s/step - loss: 0.1916 - acc: 0.9298 - val_loss: 1.1017 - val_acc: 0.6000
Epoch 7/25
97/97 [==============================] - 442s 5s/step - loss: 0.1806 - acc: 0.9360 - val_loss: 1.3826 - val_acc: 0.6176
Epoch 8/25
97/97 [==============================] - 436s 4s/step - loss: 0.1402 - a

In [ ]:
model1.save('level.h5')

In [ ]:
!tar -zcvf level.tgz level.h5

In [ ]:
ls -1 

In [ ]:
model_details = client.repository.store_model(model = 'level.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "level", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [ ]:
client.repository.download(model_id, 'level_cloud.tar.gz')

In [ ]:
model_body = load_model('level.h5')

In [ ]:
os.listdir('.')

In [ ]:
client.repository.download('model_id','body_cloud.tar.gz')

In [ ]:
client.repository.download('model1_id','level_cloud.tar.gz')